In [207]:
import pandas as pd
import numpy as np

In [256]:
df = pd.read_csv('processes2.csv')
df.head()

,Unnamed: 0,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,seats,max_power (in bph),Mileage Unit,Mileage,Engine (CC)
0,0,Maruti,2014,450000,145500,Diesel,Individual,Manual,First Owner,5,74.00,kmpl,23.40,1248
1,2,Hyundai,2010,225000,127000,Diesel,Individual,Manual,First Owner,5,90.00,kmpl,23.00,1396
2,4,Hyundai,2017,440000,45000,Petrol,Individual,Manual,First Owner,5,81.86,kmpl,20.14,1197
3,7,Toyota,2011,350000,90000,Diesel,Individual,Manual,First Owner,5,67.10,kmpl,23.59,1364
4,8,Ford,2013,200000,169000,Diesel,Individual,Manual,First Owner,5,68.10,kmpl,20.00,1399


In [257]:
print(df.shape)
print(df.columns)
df.sample(5)

(2095, 14)
Index(['Unnamed: 0', 'name', 'year', 'selling_price', 'km_driven', 'fuel',
       'seller_type', 'transmission', 'owner', 'seats', 'max_power (in bph)',
       'Mileage Unit', 'Mileage', 'Engine (CC)'],
      dtype='object')


,Unnamed: 0,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,seats,max_power (in bph),Mileage Unit,Mileage,Engine (CC)
1551,4664,Renault,2012,450000,80000,Diesel,Individual,Manual,First Owner,5,83.80,kmpl,19.87,1461
1927,5744,Maruti,2017,620000,37000,Petrol,Dealer,Manual,First Owner,5,81.80,kmpl,21.01,1197
235,755,Tata,2017,280000,80000,Diesel,Individual,Manual,First Owner,5,74.00,kmpl,22.95,1248
591,1795,Hyundai,2016,595000,60000,Diesel,Individual,Manual,Second Owner,5,88.73,kmpl,22.54,1396
1967,5848,Maruti,2013,409999,100000,Diesel,Individual,Manual,Third Owner,5,74.00,kmpl,22.90,1248


In [258]:
# Q1 = df['Mileage'].quantile(0.25)
# Q3 = df['Mileage'].quantile(0.75)
# IQR = Q3 - Q1
# lower_limit = Q1 - 1.5 * IQR
# upper_limit = Q3 + 1.5 * IQR

# df['Mileage'] = np.clip(df['Mileage'], lower_limit, upper_limit)

# Q2 = df['Price'].quantile(0.25)
# Q4 = df['Price'].quantile(0.75)
# IQR = Q4 - Q2
# lower_limit = Q2 - 1.5 * IQR
# upper_limit = Q4 + 1.5 * IQR

# df['Price'] = np.clip(df['Price'], lower_limit, upper_limit)


# Q1 = df[['Year', 'Engine Size', 'Mileage']].quantile(0.25)
# Q3 = df[['Year', 'Engine Size', 'Mileage']].quantile(0.75)
# IQR = Q3 - Q1

# lower_limit = Q1 - 1.5 * IQR
# upper_limit = Q3 + 1.5 * IQR


# df_no_outliers = df[~((df[['Year', 'Engine Size', 'Mileage']] < lower_limit) | (df[['Year', 'Engine Size', 'Mileage']] > upper_limit)).any(axis=1)]


# print(f"Data shape after removing outliers: {df_no_outliers.shape}")

In [259]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095 entries, 0 to 2094
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          2095 non-null   int64  
 1   name                2095 non-null   object 
 2   year                2095 non-null   int64  
 3   selling_price       2095 non-null   int64  
 4   km_driven           2095 non-null   int64  
 5   fuel                2095 non-null   object 
 6   seller_type         2095 non-null   object 
 7   transmission        2095 non-null   object 
 8   owner               2095 non-null   object 
 9   seats               2095 non-null   int64  
 10  max_power (in bph)  2095 non-null   float64
 11  Mileage Unit        2095 non-null   object 
 12  Mileage             2095 non-null   float64
 13  Engine (CC)         2095 non-null   int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 229.3+ KB


In [260]:
df.describe()

,Unnamed: 0,year,selling_price,km_driven,seats,max_power (in bph),Mileage,Engine (CC)
count,2095.000000,2095.000000,2.095000e+03,2095.000000,2095.000000,2095.000000,2095.000000,2095.000000
mean,3155.537947,2014.052506,4.272799e+05,67413.593795,5.096897,77.410427,21.397928,1205.078282
std,1791.039215,3.273451,2.326698e+05,41580.846333,0.457000,19.884938,1.209234,245.058696
min,0.000000,2000.000000,4.595700e+04,1000.000000,4.000000,35.000000,19.600000,624.000000
25%,1589.500000,2012.000000,2.500000e+05,35000.000000,5.000000,67.040000,20.360000,998.000000
50%,3185.000000,2014.000000,3.900000e+05,60000.000000,5.000000,74.000000,21.100000,1248.000000
75%,4709.500000,2017.000000,5.555000e+05,90000.000000,5.000000,88.730000,22.540000,1396.000000
max,6256.000000,2020.000000,1.594000e+06,270000.000000,8.000000,126.320000,23.590000,1598.000000


In [261]:
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df.isnull().sum()

Unnamed: 0            0
name                  0
year                  0
selling_price         0
km_driven             0
fuel                  0
seller_type           0
transmission          0
owner                 0
seats                 0
max_power (in bph)    0
Mileage Unit          0
Mileage               0
Engine (CC)           0
dtype: int64

In [262]:
# X.dtypes

In [263]:
df.duplicated().sum()

0

In [266]:
X=df.drop(columns=['selling_price'])
Y=df['selling_price']

print(X.dtypes)
print(X.isnull().sum())
num_cols=X.select_dtypes(include=['float64','int64']).columns
cat_cols=X.select_dtypes(include=['object']).columns
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])


Unnamed: 0              int64
name                   object
year                    int64
km_driven               int64
fuel                   object
seller_type            object
transmission           object
owner                  object
seats                   int64
max_power (in bph)    float64
Mileage Unit           object
Mileage               float64
Engine (CC)             int64
dtype: object
Unnamed: 0            0
name                  0
year                  0
km_driven             0
fuel                  0
seller_type           0
transmission          0
owner                 0
seats                 0
max_power (in bph)    0
Mileage Unit          0
Mileage               0
Engine (CC)           0
dtype: int64


In [267]:


preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)
    ]
)
X_transformed = preprocessor.fit_transform(X)

onehot_columns = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(cat_cols)
all_columns = list(num_cols) + list(onehot_columns)

X = pd.DataFrame(X_transformed, columns=all_columns)


In [268]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=75)


print(f"Training_data_shape= {X_train.shape}")
print(f"Testinf_data_shape= {X_test.shape}")

Training_data_shape= (1676, 33)
Testinf_data_shape= (419, 33)


In [269]:
from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
# from sklearn.tree import DecisionTreeRegressor
# model = model = DecisionTreeRegressor(random_state=42) 
model = RandomForestRegressor(n_estimators=200,random_state=42)
# model =LinearRegression()
model.fit(X_train,Y_train)
Y_pred = model.predict(X_test)
mse = mean_squared_error(Y_test,Y_pred)
r2 = r2_score(Y_test,Y_pred)

In [270]:
print(f"mean_squared_error:{mse}")
print(f"r2_score: {r2}")

mean_squared_error:5643873096.08375
r2_score: 0.8908857702026611
